# Dinamic web scrapping
## PitchFork album reviews

#### BeautifulSoup and Selenium are our most important libraries for this application

In [9]:
from bs4 import BeautifulSoup
from requests import get
import warnings
warnings.filterwarnings("ignore")
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import random
import datetime
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager


#### Path for Chrome executable file

In [10]:
path = "C:/Users/bfmor/Downloads/chromedriver_win32/chromedriver.exe"

#### Executing Chrome

In [11]:
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
#driver = webdriver.Chrome(path, chrome_options = options)
driver = webdriver.Chrome(ChromeDriverManager().install())
oldWindow = driver.current_window_handle

[WDM] - Current google-chrome version is 86.0.4240
[WDM] - Get LATEST driver version for 86.0.4240
[WDM] - Driver [C:\Users\bfmor\.wdm\drivers\chromedriver\win32\86.0.4240.22\chromedriver.exe] found in cache


#### Acessing URL - Pitchfork reviews

In [12]:
driver.get("https://pitchfork.com/reviews/albums/")

#### Scroll document

In [13]:
x = 0
while x < 1:
    
    try:
        anterior = driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        altura = driver.execute_script("return document.body.scrollHeight")
        
        if altura == anterior:
            break
        anterior = altura
        
        time.sleep(2)
        
    finally:
        
        time.sleep(2)
        webdriver.ActionChains(driver).send_keys(Keys.ESCAPE).perform()
        x+=1
    

#### Parses html into string

In [14]:
text = driver.page_source
parsed_content = BeautifulSoup(text, 'html.parser')
review = parsed_content.findAll('div',{'class':'review'})

#### Structuring lists and dataframe from general reviews list

In [15]:
artista = []
album = []
genero = []
autor_review = []
data_review = []

for i in review:
    
    artist = i.find('ul',{'class':'artist-list review__title-artist'}).text
    artista.append(artist)

    album_name = i.find('h2',{'class':'review__title-album'}).text
    album.append(album_name)
    
    genre = i.findAll('a')[1].text
    genero.append(genre)

    author = i.find('a',{'class':'linked display-name display-name--linked'}).text.replace('by: ','')
    autor_review.append(author)
    
    date = i.time.get('datetime')
    data_review.append(date)
    

#### Structuring lists and dataframe from each review page content (text and rating)

In [16]:
rating = []
content = []
links=[]

for i in review:
    
    link = i.a.get('href')
    links.append(link)
    
    #entra no link de cada review e pega o html
    driver.get('https://pitchfork.com' + link)
    text2 = driver.page_source
    ######################################################################################################
    
    parsed = BeautifulSoup(text2, 'html.parser')
    container = parsed.findAll('div',{'class':'review-detail'})
    time.sleep(1)
    
    #pega nota do review
    review_score = container[0].find('article', class_='review-detail__article container-fluid').find('div', class_='score-circle').text
    rating.append(review_score)
    
    #pega texto do review
    review_content = container[0].find('article', class_='review-detail__article container-fluid').find('div', class_='review-detail__text clearfix').text
    content.append(review_content)
    
    #tempo aleatório até próxima busca
    time.sleep(random.randint(0,2))

#### Inputs Each review info, content and rating into dataset "data"

In [17]:
data = pd.DataFrame({'album':album,
                      'artist':artista,
                      'reviwer':autor_review,
                      'review_date':data_review,
                      'link':link, 
                      'genero':genero,   
                      'rating':rating,
                      'review_content':content})

In [18]:
data.head(10)

,album,artist,reviwer,review_date,link,genero,rating,review_content
0,Lament,Touché Amoré,Ian Cohen,2020-10-12T05:00:00,/reviews/albums/rilo-kiley-rilo-kiley/,Best New Album,8.2,Featured Tracks:Play Track“Reminders” — Touché...
1,Soundkeeper,Gunn-Truscinski Duo,Sam Sodomsky,2020-10-12T05:00:00,/reviews/albums/rilo-kiley-rilo-kiley/,Experimental,7.5,Featured Tracks:Play Track“For Eddie Hazel” — ...
2,Nothing EP,Loraine James,Philip Sherburne,2020-10-12T05:00:00,/reviews/albums/rilo-kiley-rilo-kiley/,Electronic,7.9,Featured Tracks:Play Track“Marg” [ft. Tardast]...
3,Love Touched Time and Time Began to Sweat,Pool Holograph,Steven Arroyo,2020-10-12T05:00:00,/reviews/albums/rilo-kiley-rilo-kiley/,Rock,7.1,Featured Tracks:Play Track“Medieval Heart” — P...
4,Innercity Griots,Freestyle Fellowship,Rollie Pemberton,2020-10-11T05:00:00,/reviews/albums/rilo-kiley-rilo-kiley/,Rap,9.0,A couple decades before Kendrick Lamar galvani...
5,Hybrid Theory (20th Anniversary Edition),Linkin Park,Gabriel Szatan,2020-10-10T05:00:00,/reviews/albums/rilo-kiley-rilo-kiley/,Rock,7.6,Chester Bennington was 41 years old when he lo...
6,The Seeds of Love (Super Deluxe Edition),Tears for Fears,Alfred Soto,2020-10-10T05:00:00,/reviews/albums/rilo-kiley-rilo-kiley/,Rock,7.2,To transform from a bedsit synth-pop outfit wi...
7,Diamond Life,Sade,Clover Hope,2020-10-09T05:00:00,/reviews/albums/rilo-kiley-rilo-kiley/,Pop/R&B,9.6,"In 1984, while British new romantics like Dura..."
8,Promise,Sade,Naima Cochrane,2020-10-09T05:00:00,/reviews/albums/rilo-kiley-rilo-kiley/,Pop/R&B,9.0,"High gloss, high glam, and high production-val..."
9,Lovers Rock,Sade,Rawiya Kameir,2020-10-09T05:00:00,/reviews/albums/rilo-kiley-rilo-kiley/,Pop/R&B,8.7,"Even on Sade’s official website, you have to d..."


#### Exports final dataset into csv

In [20]:
data.to_csv('Reviews_content2.csv')

In [21]:
data.shape

(36, 8)